In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from scipy import stats

In [3]:
def grad_step(w, X, y):
    return X.T @ (sigmoid(X @ w) - y) / len(y)

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
def bce_loss(y_true, y_pred):
    return -np.mean(y_true*np.log(y_pred) + (1 - y_true)*np.log(1 - y_pred))

In [6]:
skyfall=pd.read_csv('weatherAUS_train.csv',delimiter=',')
Test=pd.read_csv('weatherAUS_test.csv',delimiter=',')
skyfall

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2013-07-22,Sydney,8.7,17.2,0.0,3.8,9.9,WNW,54.0,WNW,...,54.0,28.0,1019.4,1016.3,2.0,2.0,12.2,16.6,No,0
1,2015-02-28,Sale,15.4,25.7,0.0,NaN,NaN,W,67.0,NNE,...,84.0,66.0,1008.1,1005.1,8.0,8.0,18.8,24.3,No,1
2,2009-11-18,PerthAirport,15.2,22.2,7.2,5.2,3.3,NW,78.0,NW,...,68.0,91.0,1001.5,1000.1,7.0,8.0,20.1,16.5,Yes,1
3,2014-02-19,Richmond,19.9,26.4,0.0,2.8,NaN,SW,24.0,NNE,...,97.0,97.0,1004.6,1002.1,NaN,NaN,22.6,22.3,No,1
4,2015-10-04,Wollongong,19.3,32.9,0.0,NaN,NaN,W,48.0,NNW,...,46.0,15.0,1022.8,1020.7,NaN,NaN,22.1,32.6,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,2015-07-19,Walpole,11.6,17.0,3.0,NaN,NaN,NNW,50.0,NNW,...,88.0,90.0,1013.2,1008.9,NaN,NaN,14.0,15.0,Yes,1
101818,2015-06-25,PerthAirport,7.6,20.4,0.0,2.4,9.2,ENE,30.0,NE,...,64.0,35.0,1027.7,1024.2,0.0,1.0,12.1,19.9,No,0
101819,2016-06-05,Newcastle,13.8,17.3,96.0,NaN,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,8.0,NaN,14.5,NaN,Yes,1
101820,2011-06-13,Richmond,10.5,14.6,4.0,NaN,NaN,S,33.0,SSW,...,82.0,90.0,1024.6,1021.6,NaN,NaN,12.2,13.9,Yes,1


In [7]:
y = skyfall['RainTomorrow']
skyfall = skyfall.drop(['RainTomorrow'], axis=1)

In [8]:
skyfall['Date'] = pd.to_datetime(skyfall['Date'])
skyfall['Year'] = skyfall['Date'].dt.year
skyfall['Id'] = pd.Series(np.arange(101822))
skyfall['Month'] = skyfall['Date'].dt.month

skyfall['Day'] = skyfall['Date'].dt.day

skyfall.drop('Date', axis=1, inplace = True)

Test['Date'] = pd.to_datetime(Test['Date'])
Test['Year'] = Test['Date'].dt.year

Test['Month'] = Test['Date'].dt.month

Test['Day'] = Test['Date'].dt.day

Test.drop('Date', axis=1, inplace = True)


In [9]:
categorical = [var for var in skyfall.columns if skyfall[var].dtype=='O']
numerical = [var for var in Test.columns if Test[var].dtype!='O']

In [10]:
for df2 in [skyfall, Test]:
    df2['WindGustDir'].fillna(skyfall['WindGustDir'].mode()[0], inplace=True)
    df2['WindDir9am'].fillna(skyfall['WindDir9am'].mode()[0], inplace=True)
    df2['WindDir3pm'].fillna(skyfall['WindDir3pm'].mode()[0], inplace=True)
    df2['RainToday'].fillna(skyfall['RainToday'].mode()[0], inplace=True)

In [11]:
skyfall

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Id,Month,Day
0,Sydney,8.7,17.2,0.0,3.8,9.9,WNW,54.0,WNW,W,...,1016.3,2.0,2.0,12.2,16.6,No,2013,0,7,22
1,Sale,15.4,25.7,0.0,NaN,NaN,W,67.0,NNE,WSW,...,1005.1,8.0,8.0,18.8,24.3,No,2015,1,2,28
2,PerthAirport,15.2,22.2,7.2,5.2,3.3,NW,78.0,NW,WSW,...,1000.1,7.0,8.0,20.1,16.5,Yes,2009,2,11,18
3,Richmond,19.9,26.4,0.0,2.8,NaN,SW,24.0,NNE,WSW,...,1002.1,NaN,NaN,22.6,22.3,No,2014,3,2,19
4,Wollongong,19.3,32.9,0.0,NaN,NaN,W,48.0,NNW,W,...,1020.7,NaN,NaN,22.1,32.6,No,2015,4,10,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,Walpole,11.6,17.0,3.0,NaN,NaN,NNW,50.0,NNW,NNW,...,1008.9,NaN,NaN,14.0,15.0,Yes,2015,101817,7,19
101818,PerthAirport,7.6,20.4,0.0,2.4,9.2,ENE,30.0,NE,WNW,...,1024.2,0.0,1.0,12.1,19.9,No,2015,101818,6,25
101819,Newcastle,13.8,17.3,96.0,NaN,NaN,W,NaN,N,SE,...,NaN,8.0,NaN,14.5,NaN,Yes,2016,101819,6,5
101820,Richmond,10.5,14.6,4.0,NaN,NaN,S,33.0,SSW,SSW,...,1021.6,NaN,NaN,12.2,13.9,Yes,2011,101820,6,13


In [13]:
towns = skyfall['Location'].unique()

In [14]:
for df1 in [skyfall, Test]:
    l = len(df1)
    loc = np.array(df1['Location'])
    mon = np.array(df1['Month'])
    for town in towns:
        ind_t = np.where(loc == town)[0]
        for month in range(1, 13):
            ind_tm = [i for i in ind_t if mon[i] == month]
            for col in numerical:
                a = np.array(df1[col])
                ind_tm_1 = [i for i in ind_tm if a[i] == a[i]]
                ind_tm_0 = [i for i in ind_tm if a[i] != a[i]]
                if (len(ind_tm_1) > 0):
                    col_tm_mean = a[ind_tm_1].mean()
                else:
                    col_tm_mean = np.mean(np.array([x for x in a if x == x]))
                df1.loc[df1.index[ind_tm_0], col] = col_tm_mean

In [15]:
skyfall

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Id,Month,Day
0,Sydney,8.7,17.2,0.0,3.800000,9.900000,WNW,54.000000,WNW,W,...,1016.300000,2.000000,2.000000,12.2,16.600000,No,2013.0,0.0,7.0,22.0
1,Sale,15.4,25.7,0.0,5.544565,7.238043,W,67.000000,NNE,WSW,...,1005.100000,8.000000,8.000000,18.8,24.300000,No,2015.0,1.0,2.0,28.0
2,PerthAirport,15.2,22.2,7.2,5.200000,3.300000,NW,78.000000,NW,WSW,...,1000.100000,7.000000,8.000000,20.1,16.500000,Yes,2009.0,2.0,11.0,18.0
3,Richmond,19.9,26.4,0.0,2.800000,7.590921,SW,24.000000,NNE,WSW,...,1002.100000,6.428571,5.441176,22.6,22.300000,No,2014.0,3.0,2.0,19.0
4,Wollongong,19.3,32.9,0.0,5.439165,7.590921,W,48.000000,NNW,W,...,1020.700000,6.125000,5.792208,22.1,32.600000,No,2015.0,4.0,10.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,Walpole,11.6,17.0,3.0,5.456674,7.627164,NNW,50.000000,NNW,NNW,...,1008.900000,4.582266,4.593913,14.0,15.000000,Yes,2015.0,101817.0,7.0,19.0
101818,PerthAirport,7.6,20.4,0.0,2.400000,9.200000,ENE,30.000000,NE,WNW,...,1024.200000,0.000000,1.000000,12.1,19.900000,No,2015.0,101818.0,6.0,25.0
101819,Newcastle,13.8,17.3,96.0,5.426587,7.621714,W,40.070905,N,SE,...,1015.250857,8.000000,4.536232,14.5,16.781159,Yes,2016.0,101819.0,6.0,5.0
101820,Richmond,10.5,14.6,4.0,2.044595,7.590921,S,33.000000,SSW,SSW,...,1021.600000,6.755556,6.424242,12.2,13.900000,Yes,2011.0,101820.0,6.0,13.0


In [438]:
skyfall['Evaporation'].fillna(value=skyfall['Evaporation'].mean(), inplace=True)
skyfall['MinTemp'].fillna(value=skyfall['MinTemp'].mean(), inplace=True)
skyfall['MaxTemp'].fillna(value=skyfall['MaxTemp'].mean(), inplace=True)
skyfall['Rainfall'].fillna(value=skyfall['Rainfall'].mean(), inplace=True)
skyfall['Sunshine'].fillna(value=skyfall['Sunshine'].mean(), inplace=True)
skyfall['WindGustSpeed'].fillna(value=skyfall['WindGustSpeed'].mean(), inplace=True)
skyfall['WindSpeed3pm'].fillna(value=skyfall['WindSpeed3pm'].mean(), inplace=True)
skyfall['WindSpeed9am'].fillna(value=skyfall['WindSpeed9am'].mean(), inplace=True)
skyfall['Cloud9am'].fillna(value=skyfall['Cloud9am'].mean(), inplace=True)
skyfall['Cloud3pm'].fillna(value=skyfall['Cloud3pm'].mean(), inplace=True)
skyfall['Temp9am'].fillna(value=skyfall['Temp9am'].mean(), inplace=True)
skyfall['Temp3pm'].fillna(value=skyfall['Temp3pm'].mean(), inplace=True)
skyfall['Pressure9am'].fillna(value=skyfall['Pressure9am'].mean(), inplace=True)
skyfall['Humidity3pm'].fillna(value=skyfall['Humidity3pm'].mean(), inplace=True)
skyfall['Humidity9am'].fillna(value=skyfall['Humidity9am'].mean(), inplace=True)
skyfall['Pressure3pm'].fillna(value=skyfall['Pressure3pm'].mean(), inplace=True)

Test['Evaporation'].fillna(value=skyfall['Evaporation'].mean(), inplace=True)
Test['MinTemp'].fillna(value=skyfall['MinTemp'].mean(), inplace=True)
Test['MaxTemp'].fillna(value=skyfall['MaxTemp'].mean(), inplace=True)
Test['Rainfall'].fillna(value=skyfall['Rainfall'].mean(), inplace=True)
Test['Sunshine'].fillna(value=skyfall['Sunshine'].mean(), inplace=True)
Test['WindGustSpeed'].fillna(value=skyfall['WindGustSpeed'].mean(), inplace=True)
Test['WindSpeed3pm'].fillna(value=skyfall['WindSpeed3pm'].mean(), inplace=True)
Test['WindSpeed9am'].fillna(value=skyfall['WindSpeed9am'].mean(), inplace=True)
Test['Cloud9am'].fillna(value=skyfall['Cloud9am'].mean(), inplace=True)
Test['Cloud3pm'].fillna(value=skyfall['Cloud3pm'].mean(), inplace=True)
Test['Temp9am'].fillna(value=skyfall['Temp9am'].mean(), inplace=True)
Test['Temp3pm'].fillna(value=skyfall['Temp3pm'].mean(), inplace=True)
Test['Pressure9am'].fillna(value=skyfall['Pressure9am'].mean(), inplace=True)
Test['Humidity3pm'].fillna(value=skyfall['Humidity3pm'].mean(), inplace=True)
Test['Humidity9am'].fillna(value=skyfall['Humidity9am'].mean(), inplace=True)
Test['Pressure3pm'].fillna(value=skyfall['Pressure3pm'].mean(), inplace=True)

In [16]:
skyfall = pd.concat([skyfall[numerical],skyfall[['RainToday']],
                     pd.get_dummies(skyfall.Location), 
                     pd.get_dummies(skyfall.WindGustDir),
                     pd.get_dummies(skyfall.WindDir9am),
                     pd.get_dummies(skyfall.WindDir3pm)], axis=1)

In [17]:
Test = pd.concat([Test[numerical],Test[['RainToday']],
                     pd.get_dummies(Test.Location), 
                     pd.get_dummies(Test.WindGustDir),
                     pd.get_dummies(Test.WindDir9am),
                     pd.get_dummies(Test.WindDir3pm)], axis=1)

In [18]:
skyfall

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,0.0,8.7,17.2,0.0,3.800000,9.900000,54.000000,17.0,26.000000,54.0,...,0,0,0,0,0,0,0,1,0,0
1,1.0,15.4,25.7,0.0,5.544565,7.238043,67.000000,11.0,9.000000,84.0,...,0,0,0,0,0,0,0,0,0,1
2,2.0,15.2,22.2,7.2,5.200000,3.300000,78.000000,33.0,28.000000,68.0,...,0,0,0,0,0,0,0,0,0,1
3,3.0,19.9,26.4,0.0,2.800000,7.590921,24.000000,13.0,9.000000,97.0,...,0,0,0,0,0,0,0,0,0,1
4,4.0,19.3,32.9,0.0,5.439165,7.590921,48.000000,19.0,22.000000,46.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,101817.0,11.6,17.0,3.0,5.456674,7.627164,50.000000,15.0,13.000000,88.0,...,1,0,0,0,0,0,0,0,0,0
101818,101818.0,7.6,20.4,0.0,2.400000,9.200000,30.000000,9.0,17.000000,64.0,...,0,0,0,0,0,0,0,0,1,0
101819,101819.0,13.8,17.3,96.0,5.426587,7.621714,40.070905,0.0,7.123188,100.0,...,0,0,0,1,0,0,0,0,0,0
101820,101820.0,10.5,14.6,4.0,2.044595,7.590921,33.000000,20.0,19.000000,82.0,...,0,0,0,0,0,1,0,0,0,0


Изменение столбца RainToday

In [19]:
a=np.ones(len(skyfall['RainToday']))    
b=np.zeros(len(skyfall['RainToday']))    
skyfall['RainToday']=np.where(skyfall['RainToday']=='Yes',a,b)
skyfall['RainToday']=pd.to_numeric(skyfall['RainToday'])

a=np.ones(len(Test['RainToday']))    
b=np.zeros(len(Test['RainToday']))    
Test['RainToday']=np.where(Test['RainToday']=='Yes',a,b)
Test['RainToday']=pd.to_numeric(Test['RainToday'])
skyfall.tail(20)

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
101802,101802.0,0.8,27.2,0.0,5.456651,7.626995,46.000000,26.0,26.000000,38.0,...,0,1,0,0,0,0,0,0,0,0
101803,101803.0,11.9,21.0,0.0,5.400000,7.500000,46.000000,22.0,30.000000,80.0,...,0,0,1,0,0,0,0,0,0,0
101804,101804.0,16.1,20.9,1.5,3.801183,6.459333,54.000000,24.0,17.000000,80.0,...,0,0,0,0,0,0,1,0,0,0
101805,101805.0,7.3,11.4,4.8,1.800000,7.900000,28.000000,11.0,13.000000,97.0,...,0,0,0,0,0,0,0,1,0,0
101806,101806.0,15.2,19.1,2.8,4.000000,1.300000,37.283333,15.0,6.000000,79.0,...,0,0,1,0,0,0,0,0,0,0
101807,101807.0,11.3,20.3,0.0,2.800000,3.700000,30.000000,6.0,4.000000,88.0,...,0,0,0,0,0,0,0,0,1,0
101808,101808.0,14.4,23.1,0.0,5.439165,7.590921,24.000000,4.0,13.000000,60.0,...,0,0,0,0,0,0,0,0,0,0
101809,101809.0,2.3,16.9,0.0,5.454656,7.626995,15.000000,6.0,7.000000,91.0,...,0,0,0,0,0,0,0,1,0,0
101810,101810.0,10.8,18.7,0.0,6.000000,8.900000,31.000000,0.0,13.000000,74.0,...,0,0,0,0,1,0,0,0,0,0
101811,101811.0,6.8,17.7,0.4,3.400000,9.700000,72.000000,6.0,30.000000,68.0,...,1,0,0,0,0,0,0,0,0,0


In [20]:
print(round(skyfall[numerical].describe()),2)

             Id   MinTemp   MaxTemp  Rainfall  Evaporation  Sunshine  \
count  101822.0  101822.0  101822.0  101822.0     101822.0  101822.0   
mean    50910.0      12.0      23.0       2.0          5.0       8.0   
std     29394.0       6.0       7.0       8.0          3.0       3.0   
min         0.0      -8.0      -5.0       0.0          0.0       0.0   
25%     25455.0       8.0      18.0       0.0          4.0       7.0   
50%     50910.0      12.0      23.0       0.0          5.0       8.0   
75%     76366.0      17.0      28.0       1.0          6.0       9.0   
max    101821.0      32.0      48.0     368.0        145.0      14.0   

       WindGustSpeed  WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  \
count       101822.0      101822.0      101822.0     101822.0     101822.0   
mean            40.0          14.0          19.0         69.0         52.0   
std             13.0           9.0           9.0         19.0         21.0   
min              7.0           0.0     

In [21]:
print(round(Test[numerical].describe()),2)

            Id  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
count  43638.0  43638.0  43638.0   43638.0      43638.0   43638.0   
mean   21818.0     12.0     23.0       2.0          5.0       8.0   
std    12597.0      6.0      7.0       9.0          3.0       3.0   
min        0.0     -8.0     -3.0       0.0          0.0       0.0   
25%    10909.0      8.0     18.0       0.0          4.0       7.0   
50%    21818.0     12.0     23.0       0.0          5.0       8.0   
75%    32728.0     17.0     28.0       1.0          6.0       9.0   
max    43637.0     34.0     47.0     371.0         82.0      14.0   

       WindGustSpeed  WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  \
count        43638.0       43638.0       43638.0      43638.0      43638.0   
mean            40.0          14.0          19.0         69.0         52.0   
std             13.0           9.0           9.0         19.0         21.0   
min              6.0           0.0           0.0          0.0     

In [22]:
IQR = skyfall.Rainfall.quantile(0.75) - skyfall.Rainfall.quantile(0.25)
Lower_fence = skyfall.Rainfall.quantile(0.25) - (IQR * 3)
Upper_fence = skyfall.Rainfall.quantile(0.75) + (IQR * 3)
print('skyfall Rainfall outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

skyfall Rainfall outliers are values < -3.0 or > 4.0


In [23]:
IQR = Test.Rainfall.quantile(0.75) - Test.Rainfall.quantile(0.25)
Lower_fence = Test.Rainfall.quantile(0.25) - (IQR * 3)
Upper_fence = Test.Rainfall.quantile(0.75) + (IQR * 3)
print('Test Rainfall outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

Test Rainfall outliers are values < -2.7 or > 3.6


In [24]:
IQR = skyfall.Evaporation.quantile(0.75) - skyfall.Evaporation.quantile(0.25)
Lower_fence = skyfall.Evaporation.quantile(0.25) - (IQR * 3)
Upper_fence = skyfall.Evaporation.quantile(0.75) + (IQR * 3)
print('skyfall Evaporation outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

skyfall Evaporation outliers are values < -3.599999999999999 or > 13.2


In [25]:
IQR = Test.Evaporation.quantile(0.75) - Test.Evaporation.quantile(0.25)
Lower_fence = Test.Evaporation.quantile(0.25) - (IQR * 3)
Upper_fence = Test.Evaporation.quantile(0.75) + (IQR * 3)
print('Test Evaporation outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

Test Evaporation outliers are values < -3.599999999999999 or > 13.2


In [26]:
IQR = skyfall.WindSpeed9am.quantile(0.75) - skyfall.WindSpeed9am.quantile(0.25)
Lower_fence = skyfall.WindSpeed9am.quantile(0.25) - (IQR * 3)
Upper_fence = skyfall.WindSpeed9am.quantile(0.75) + (IQR * 3)
print('skyfall WindSpeed9am outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

skyfall WindSpeed9am outliers are values < -29.0 or > 55.0


In [27]:
IQR = Test.WindSpeed9am.quantile(0.75) - Test.WindSpeed9am.quantile(0.25)
Lower_fence = Test.WindSpeed9am.quantile(0.25) - (IQR * 3)
Upper_fence = Test.WindSpeed9am.quantile(0.75) + (IQR * 3)
print('Test WindSpeed9am outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

Test WindSpeed9am outliers are values < -29.0 or > 55.0


In [28]:
IQR = skyfall.WindSpeed3pm.quantile(0.75) - skyfall.WindSpeed3pm.quantile(0.25)
Lower_fence = skyfall.WindSpeed3pm.quantile(0.25) - (IQR * 3)
Upper_fence = skyfall.WindSpeed3pm.quantile(0.75) + (IQR * 3)
print('skyfall WindSpeed3pm outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

skyfall WindSpeed3pm outliers are values < -20.0 or > 57.0


In [29]:
IQR = Test.WindSpeed3pm.quantile(0.75) - Test.WindSpeed3pm.quantile(0.25)
Lower_fence = Test.WindSpeed3pm.quantile(0.25) - (IQR * 3)
Upper_fence = Test.WindSpeed3pm.quantile(0.75) + (IQR * 3)
print('Test WindSpeed3pm outliers are values < {lowerboundary} or > {upperboundary}'.format(lowerboundary=Lower_fence, upperboundary=Upper_fence))

Test WindSpeed3pm outliers are values < -20.0 or > 57.0


In [30]:
def max_value(df3, variable, top):
    return np.where(df3[variable]>top, top, df3[variable])
def min_value(df3, variable, low):
    return np.where(df3[variable]<low, low, df3[variable])

for df3 in [skyfall, Test]:
    df3['Rainfall'] = max_value(df3, 'Rainfall', 4.0)
    df3['Evaporation'] = max_value(df3, 'Evaporation', 9.280406676678158)
    df3['Evaporation'] = min_value(df3, 'Evaporation', 0.38969499249138106)
    df3['WindSpeed9am'] = max_value(df3, 'WindSpeed9am', 55)
    df3['WindSpeed3pm'] = max_value(df3, 'WindSpeed3pm', 57)

Нормировка

In [31]:
cols = skyfall.columns

In [32]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

skyfall = scaler.fit_transform(skyfall)

Test = scaler.transform(Test)

In [33]:
skyfall = pd.DataFrame(skyfall, columns=[cols])
Test = pd.DataFrame(Test, columns=[cols])

In [34]:
skyfall

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,0.000000,0.425743,0.415879,0.00,0.383581,0.692308,0.367188,0.309091,0.456140,0.535354,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.000010,0.591584,0.576560,0.00,0.579804,0.506157,0.468750,0.200000,0.157895,0.838384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.000020,0.586634,0.510397,1.00,0.541048,0.230769,0.554688,0.600000,0.491228,0.676768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.000029,0.702970,0.589792,0.00,0.271104,0.530834,0.132812,0.236364,0.157895,0.969697,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.000039,0.688119,0.712665,0.00,0.567949,0.530834,0.320312,0.345455,0.385965,0.454545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,0.999961,0.497525,0.412098,0.75,0.569918,0.533368,0.335938,0.272727,0.228070,0.878788,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101818,0.999971,0.398515,0.476371,0.00,0.226113,0.643357,0.179688,0.163636,0.298246,0.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
101819,0.999980,0.551980,0.417769,1.00,0.566534,0.532987,0.258366,0.000000,0.124968,1.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
101820,0.999990,0.470297,0.366730,1.00,0.186138,0.530834,0.203125,0.363636,0.333333,0.818182,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Корреляционный анализ ниже

In [35]:
corr = skyfall.corr(method='pearson')
corr

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
Id,1.000000,0.002308,0.001170,0.004926,0.001815,0.002051,0.004705,0.003599,0.003198,-0.003613,...,-0.005550,-0.000786,0.005513,-0.000543,-0.002009,0.002647,0.009953,-0.001204,-0.001540,0.001405
MinTemp,0.002308,1.000000,0.737682,0.063575,0.458144,0.081109,0.173180,0.170981,0.173464,-0.236858,...,-0.068742,-0.064211,-0.005818,0.054078,0.024471,-0.035274,-0.046363,-0.076232,-0.047022,-0.061829
MaxTemp,0.001170,0.737682,1.000000,-0.227451,0.552240,0.367320,0.066915,0.013080,0.051437,-0.504774,...,-0.006634,-0.009741,-0.054508,0.002921,-0.014719,-0.059764,-0.056274,-0.066059,-0.027267,-0.061011
Rainfall,0.004926,0.063575,-0.227451,1.000000,-0.167684,-0.251297,0.156993,0.104286,0.076564,0.366536,...,-0.028146,0.002494,0.030538,0.014304,0.012759,0.044602,0.032940,0.042401,0.034510,0.041882
Evaporation,0.001815,0.458144,0.552240,-0.167684,1.000000,0.390313,0.173050,0.147709,0.118957,-0.437060,...,-0.036351,-0.026310,-0.016624,0.019383,0.017413,-0.014693,-0.002198,-0.031800,-0.027109,-0.018176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SSW,0.002647,-0.035274,-0.059764,0.044602,-0.014693,-0.010062,-0.002996,0.012678,-0.002286,0.026262,...,-0.058094,-0.061162,-0.065643,-0.082875,-0.063796,1.000000,-0.063599,-0.066089,-0.062310,-0.064096
SW,0.009953,-0.046363,-0.056274,0.032940,-0.002198,0.022293,0.005336,-0.003524,0.007577,-0.006135,...,-0.062667,-0.065976,-0.070810,-0.089398,-0.068818,-0.063599,1.000000,-0.071291,-0.067215,-0.069141
W,-0.001204,-0.076232,-0.066059,0.042401,-0.031800,-0.017701,0.086603,0.028798,0.062464,0.000060,...,-0.065121,-0.068560,-0.073583,-0.092899,-0.071512,-0.066089,-0.071291,1.000000,-0.069847,-0.071849
WNW,-0.001540,-0.047022,-0.027267,0.034510,-0.027109,-0.014283,0.086367,0.016615,0.056586,0.000242,...,-0.061397,-0.064640,-0.069376,-0.087587,-0.067423,-0.062310,-0.067215,-0.069847,1.000000,-0.067741


In [36]:
towns = [] #список всех городов
for i in range(49):
    towns.append(list(skyfall)[i+21])

In [37]:
names = list(skyfall)
names

[('Id',),
 ('MinTemp',),
 ('MaxTemp',),
 ('Rainfall',),
 ('Evaporation',),
 ('Sunshine',),
 ('WindGustSpeed',),
 ('WindSpeed9am',),
 ('WindSpeed3pm',),
 ('Humidity9am',),
 ('Humidity3pm',),
 ('Pressure9am',),
 ('Pressure3pm',),
 ('Cloud9am',),
 ('Cloud3pm',),
 ('Temp9am',),
 ('Temp3pm',),
 ('Year',),
 ('Month',),
 ('Day',),
 ('RainToday',),
 ('Adelaide',),
 ('Albany',),
 ('Albury',),
 ('AliceSprings',),
 ('BadgerysCreek',),
 ('Ballarat',),
 ('Bendigo',),
 ('Brisbane',),
 ('Cairns',),
 ('Canberra',),
 ('Cobar',),
 ('CoffsHarbour',),
 ('Dartmoor',),
 ('Darwin',),
 ('GoldCoast',),
 ('Hobart',),
 ('Katherine',),
 ('Launceston',),
 ('Melbourne',),
 ('MelbourneAirport',),
 ('Mildura',),
 ('Moree',),
 ('MountGambier',),
 ('MountGinini',),
 ('Newcastle',),
 ('Nhil',),
 ('NorahHead',),
 ('NorfolkIsland',),
 ('Nuriootpa',),
 ('PearceRAAF',),
 ('Penrith',),
 ('Perth',),
 ('PerthAirport',),
 ('Portland',),
 ('Richmond',),
 ('Sale',),
 ('SalmonGums',),
 ('Sydney',),
 ('SydneyAirport',),
 ('Townsvil

Найдём города, наиболее близкие к морю (near_sea) и наиболее далекие от моря (inland)

In [38]:
print(corr.loc[towns[0],'Rainfall'].values[0][0])
print(corr.loc[towns[0],'WindSpeed3pm'].values[0][0])
print(corr.loc[towns[0],'Evaporation'].values[0][0])
print(corr.loc[towns[0],'Cloud3pm'].values[0][0])
print(corr.loc[towns[0],'Humidity3pm'].values[0][0])
print(corr.loc[towns[0],'Pressure3pm'].values[0][0])

-0.003940539260669753
-0.05372280724596881
-0.00013696588572003577
0.00045742062478891927
-0.04828516752518768
0.03277628203689041


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [39]:
near_sea =[]
for i in towns:
    #print(i,corr.loc[i,'Rainfall'].values[0][0])
    if( (corr.loc[i,'Cloud3pm'].values[0][0]) > 0.01 and (corr.loc[i,'Humidity3pm'].values[0][0]) > 0.01 and (corr.loc[i,'Pressure3pm'].values[0][0]) > 0.01):
        near_sea.append(i)

In [40]:
near_sea

[('Albany',),
 ('Ballarat',),
 ('MountGambier',),
 ('NorfolkIsland',),
 ('Portland',),
 ('Sale',),
 ('Watsonia',),
 ('Wollongong',)]

In [41]:
#(corr.loc[i,'Cloud3pm'].values[0][0]) < -0.01 and 
#and (corr.loc[i,'Pressure3pm'].values[0][0]) < -0.01

In [42]:
inland =[]
for i in towns:
    #print(i,corr.loc[i,'Rainfall'].values[0][0])
    if( (corr.loc[i,'Humidity3pm'].values[0][0]) < -0.01 and (corr.loc[i,'Pressure3pm'].values[0][0]) < -0.01 ):
        inland.append(i)

In [43]:
inland

[('AliceSprings',), ('Katherine',), ('Uluru',)]

Добавим их в skyfall и Test с соотв. значениями по параметру близости к морю (1 и -1 соотв., а иначе 0)

In [44]:
skyfall['NearSea'] = pd.Series(np.zeros(101822))

In [45]:
for i in near_sea:
    a = np.where(np.array(skyfall[i]) != 0)[0]
    for j in range(len(a)):
        skyfall.iat[a[j],118] = 1.0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\zmqshell.py:536: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [46]:
for i in inland:
    a = np.where(np.array(skyfall[i]) != 0)[0]
    for j in range(len(a)):
        skyfall.iat[a[j],118] = -1.0

Ниже для Test

In [47]:
Test['NearSea'] = pd.Series(np.zeros(43638))

In [48]:
for i in near_sea:
    a = np.where(np.array(Test[i]) != 0)[0]
    for j in range(len(a)):
        Test.iat[a[j],118] = 1.0

In [49]:
for i in inland:
    a = np.where(np.array(Test[i]) != 0)[0]
    for j in range(len(a)):
        Test.iat[a[j],118] = -1.0

In [50]:
skyfall

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,NW,S,SE,SSE,SSW,SW,W,WNW,WSW,NearSea
0,0.000000,0.425743,0.415879,0.00,0.383581,0.692308,0.367188,0.309091,0.456140,0.535354,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.000010,0.591584,0.576560,0.00,0.579804,0.506157,0.468750,0.200000,0.157895,0.838384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,0.000020,0.586634,0.510397,1.00,0.541048,0.230769,0.554688,0.600000,0.491228,0.676768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000029,0.702970,0.589792,0.00,0.271104,0.530834,0.132812,0.236364,0.157895,0.969697,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000039,0.688119,0.712665,0.00,0.567949,0.530834,0.320312,0.345455,0.385965,0.454545,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,0.999961,0.497525,0.412098,0.75,0.569918,0.533368,0.335938,0.272727,0.228070,0.878788,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101818,0.999971,0.398515,0.476371,0.00,0.226113,0.643357,0.179688,0.163636,0.298246,0.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
101819,0.999980,0.551980,0.417769,1.00,0.566534,0.532987,0.258366,0.000000,0.124968,1.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101820,0.999990,0.470297,0.366730,1.00,0.186138,0.530834,0.203125,0.363636,0.333333,0.818182,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Все эти операции добавили нам одну единичку, улучшив рез-тат на 15 стотысячных

Попробуем с засушливостью, параметры: солнечный свет, температура в 3, амплитуда максимальной и минимальной

In [51]:
a = np.array(skyfall['MinTemp']).reshape(1,len(np.array(skyfall['MinTemp'])))
b = np.array(skyfall['MaxTemp']).reshape(1,len(np.array(skyfall['MaxTemp'])))

In [52]:
#print(a,b)
c = a-b
skyfall['DeltaTemp'] = pd.Series(c[0])

In [53]:
skyfall

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,S,SE,SSE,SSW,SW,W,WNW,WSW,NearSea,DeltaTemp
0,0.000000,0.425743,0.415879,0.00,0.383581,0.692308,0.367188,0.309091,0.456140,0.535354,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.009864
1,0.000010,0.591584,0.576560,0.00,0.579804,0.506157,0.468750,0.200000,0.157895,0.838384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.015025
2,0.000020,0.586634,0.510397,1.00,0.541048,0.230769,0.554688,0.600000,0.491228,0.676768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.076237
3,0.000029,0.702970,0.589792,0.00,0.271104,0.530834,0.132812,0.236364,0.157895,0.969697,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.113178
4,0.000039,0.688119,0.712665,0.00,0.567949,0.530834,0.320312,0.345455,0.385965,0.454545,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.024547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,0.999961,0.497525,0.412098,0.75,0.569918,0.533368,0.335938,0.272727,0.228070,0.878788,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085426
101818,0.999971,0.398515,0.476371,0.00,0.226113,0.643357,0.179688,0.163636,0.298246,0.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.077856
101819,0.999980,0.551980,0.417769,1.00,0.566534,0.532987,0.258366,0.000000,0.124968,1.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.134211
101820,0.999990,0.470297,0.366730,1.00,0.186138,0.530834,0.203125,0.363636,0.333333,0.818182,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.103567


In [54]:
corr = skyfall.corr(method='pearson')
corr

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,S,SE,SSE,SSW,SW,W,WNW,WSW,NearSea,DeltaTemp
Id,1.000000,0.002308,0.001170,0.004926,0.001815,0.002051,0.004705,0.003599,0.003198,-0.003613,...,0.005513,-0.000543,-0.002009,0.002647,0.009953,-0.001204,-0.001540,0.001405,0.008205,0.001920
MinTemp,0.002308,1.000000,0.737682,0.063575,0.458144,0.081109,0.173180,0.170981,0.173464,-0.236858,...,-0.005818,0.054078,0.024471,-0.035274,-0.046363,-0.076232,-0.047022,-0.061829,-0.113199,0.545604
MaxTemp,0.001170,0.737682,1.000000,-0.227451,0.552240,0.367320,0.066915,0.013080,0.051437,-0.504774,...,-0.054508,0.002921,-0.014719,-0.059764,-0.056274,-0.066059,-0.027267,-0.061011,-0.274720,-0.163320
Rainfall,0.004926,0.063575,-0.227451,1.000000,-0.167684,-0.251297,0.156993,0.104286,0.076564,0.366536,...,0.030538,0.014304,0.012759,0.044602,0.032940,0.042401,0.034510,0.041882,0.072295,0.375229
Evaporation,0.001815,0.458144,0.552240,-0.167684,1.000000,0.390313,0.173050,0.147709,0.118957,-0.437060,...,-0.016624,0.019383,0.017413,-0.014693,-0.002198,-0.031800,-0.027109,-0.018176,-0.205267,-0.016008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W,-0.001204,-0.076232,-0.066059,0.042401,-0.031800,-0.017701,0.086603,0.028798,0.062464,0.000060,...,-0.073583,-0.092899,-0.071512,-0.066089,-0.071291,1.000000,-0.069847,-0.071849,0.044492,-0.029398
WNW,-0.001540,-0.047022,-0.027267,0.034510,-0.027109,-0.014283,0.086367,0.016615,0.056586,0.000242,...,-0.069376,-0.087587,-0.067423,-0.062310,-0.067215,-0.069847,1.000000,-0.067741,-0.013952,-0.034866
WSW,0.001405,-0.061829,-0.061011,0.041882,-0.018176,0.007817,0.047707,0.016875,0.043164,-0.005454,...,-0.071364,-0.090098,-0.069356,-0.064096,-0.069141,-0.071849,-0.067741,1.000000,0.038916,-0.014618
NearSea,0.008205,-0.113199,-0.274720,0.072295,-0.205267,-0.142021,0.062734,0.069995,0.107907,0.252509,...,0.023300,-0.012582,-0.016306,0.035361,0.034068,0.044492,-0.013952,0.038916,1.000000,0.175589


In [55]:
dry_city =[]
for i in towns:
    if( (corr.loc[i,'Sunshine'].values[0][0]) > 0.01 and (corr.loc[i,'DeltaTemp'].values[0][0]) > 0.01 and (corr.loc[i,'Temp3pm'].values[0][0]) > 0.01):
        dry_city.append(i)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [56]:
dry_city

[('Brisbane',), ('Darwin',), ('Townsville',)]

In [57]:
not_dry =[]
for i in towns:
    #print(i,corr.loc[i,'Rainfall'].values[0][0])
    if( (corr.loc[i,'Sunshine'].values[0][0]) < -0.01 and (corr.loc[i,'DeltaTemp'].values[0][0]) < -0.01 and (corr.loc[i,'Temp3pm'].values[0][0]) < -0.01 ):
        not_dry.append(i)

In [58]:
not_dry

[('Dartmoor',), ('MountGambier',), ('Sale',)]

In [59]:
skyfall['Tropics'] = pd.Series(np.zeros(101822))
for i in dry_city:
    a = np.where(np.array(skyfall[i]) != 0)[0]
    for j in range(len(a)):
        skyfall.iat[a[j],120] = 1.0
for i in not_dry:
    a = np.where(np.array(skyfall[i]) != 0)[0]
    for j in range(len(a)):
        skyfall.iat[a[j],120] = -1.0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\zmqshell.py:536: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [60]:
a = np.array(Test['MinTemp']).reshape(1,len(np.array(Test['MinTemp'])))
b = np.array(Test['MaxTemp']).reshape(1,len(np.array(Test['MaxTemp'])))
c = a-b
Test['DeltaTemp'] = pd.Series(c[0])

In [61]:
Test['Tropics'] = pd.Series(np.zeros(43638))
for i in dry_city:
    a = np.where(np.array(Test[i]) != 0)[0]
    for j in range(len(a)):
        Test.iat[a[j],120] = 1.0
for i in not_dry:
    a = np.where(np.array(Test[i]) != 0)[0]
    for j in range(len(a)):
        Test.iat[a[j],120] = -1.0

In [62]:
Test

,Id,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,SE,SSE,SSW,SW,W,WNW,WSW,NearSea,DeltaTemp,Tropics
0,0.000000,0.398515,0.621928,0.00,0.257044,0.418221,0.235795,0.127273,0.157895,0.979798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.223413,-1.0
1,0.000010,0.631188,0.637051,0.00,0.567681,0.534469,0.203125,0.000000,0.192982,0.747475,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005863,0.0
2,0.000020,0.670792,0.519849,0.00,0.721011,0.384615,0.234375,0.163636,0.456140,0.707071,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150943,0.0
3,0.000029,0.779703,0.686200,1.00,0.316094,0.534308,0.390625,0.436364,0.491228,0.929293,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.093503,0.0
4,0.000039,0.673267,0.584121,0.00,0.565823,0.531373,0.148438,0.236364,0.333333,0.727273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089146,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43633,0.428527,0.381188,0.355388,0.00,0.068645,0.306262,0.187500,0.236364,0.298246,0.949495,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.025801,-1.0
43634,0.428536,0.452970,0.517958,0.05,0.569393,0.534795,0.187500,0.127273,0.298246,0.545455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.064988,0.0
43635,0.428546,0.611386,0.572779,0.00,0.320112,0.467876,0.203125,0.036364,0.228070,0.686869,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038607,0.0
43636,0.428556,0.448020,0.387524,0.75,0.541048,0.139860,0.265625,0.000000,0.350877,0.919192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.060496,0.0


In [63]:
del skyfall['MinTemp']

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [64]:
del Test['MinTemp']

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [65]:
skyfall

,Id,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,SE,SSE,SSW,SW,W,WNW,WSW,NearSea,DeltaTemp,Tropics
0,0.000000,0.415879,0.00,0.383581,0.692308,0.367188,0.309091,0.456140,0.535354,0.280000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.009864,0.0
1,0.000010,0.576560,0.00,0.579804,0.506157,0.468750,0.200000,0.157895,0.838384,0.660000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.015025,-1.0
2,0.000020,0.510397,1.00,0.541048,0.230769,0.554688,0.600000,0.491228,0.676768,0.910000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.076237,0.0
3,0.000029,0.589792,0.00,0.271104,0.530834,0.132812,0.236364,0.157895,0.969697,0.970000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.113178,0.0
4,0.000039,0.712665,0.00,0.567949,0.530834,0.320312,0.345455,0.385965,0.454545,0.150000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-0.024547,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101817,0.999961,0.412098,0.75,0.569918,0.533368,0.335938,0.272727,0.228070,0.878788,0.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085426,0.0
101818,0.999971,0.476371,0.00,0.226113,0.643357,0.179688,0.163636,0.298246,0.636364,0.350000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.077856,0.0
101819,0.999980,0.417769,1.00,0.566534,0.532987,0.258366,0.000000,0.124968,1.000000,0.620657,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.134211,0.0
101820,0.999990,0.366730,1.00,0.186138,0.530834,0.203125,0.363636,0.333333,0.818182,0.900000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.103567,0.0


Удалил MinTemp, потому что и Min, и Max, и delta -- слишком много и коррелируют друг с другом

Давайте теперь возьмем разности у всех параметров с двумя временами

In [66]:
a = np.array(skyfall['WindSpeed3pm']).reshape(1,len(np.array(skyfall['WindSpeed3pm'])))
b = np.array(skyfall['WindSpeed9am']).reshape(1,len(np.array(skyfall['WindSpeed9am'])))
c = a-b
skyfall['SpeedFrac'] = pd.Series(c[0])

In [67]:
a = np.array(skyfall['Humidity3pm']).reshape(1,len(np.array(skyfall['Humidity3pm'])))
b = np.array(skyfall['Humidity9am']).reshape(1,len(np.array(skyfall['Humidity9am'])))
c = a-b
skyfall['HumidityFrac'] = pd.Series(c[0])

In [68]:
a = np.array(skyfall['Pressure3pm']).reshape(1,len(np.array(skyfall['Pressure3pm'])))
b = np.array(skyfall['Pressure9am']).reshape(1,len(np.array(skyfall['Pressure9am'])))
c = a-b
skyfall['PressureFrac'] = pd.Series(c[0])

In [69]:
a = np.array(skyfall['Cloud3pm']).reshape(1,len(np.array(skyfall['Cloud3pm'])))
b = np.array(skyfall['Cloud9am']).reshape(1,len(np.array(skyfall['Cloud9am'])))
c = a-b
skyfall['CloudFrac'] = pd.Series(c[0])

In [70]:
a = np.array(skyfall['Temp3pm']).reshape(1,len(np.array(skyfall['Temp3pm'])))
b = np.array(skyfall['Temp9am']).reshape(1,len(np.array(skyfall['Temp9am'])))
c = a-b
skyfall['TempFrac'] = pd.Series(c[0])

In [71]:
a = np.array(Test['WindSpeed3pm']).reshape(1,len(np.array(Test['WindSpeed3pm'])))
b = np.array(Test['WindSpeed9am']).reshape(1,len(np.array(Test['WindSpeed9am'])))
c = a-b
Test['SpeedFrac'] = pd.Series(c[0])
a = np.array(Test['Humidity3pm']).reshape(1,len(np.array(Test['Humidity3pm'])))
b = np.array(Test['Humidity9am']).reshape(1,len(np.array(Test['Humidity9am'])))
c = a-b
Test['HumidityFrac'] = pd.Series(c[0])
a = np.array(Test['Pressure3pm']).reshape(1,len(np.array(Test['Pressure3pm'])))
b = np.array(Test['Pressure9am']).reshape(1,len(np.array(Test['Pressure9am'])))
c = a-b
Test['PressureFrac'] = pd.Series(c[0])
a = np.array(Test['Cloud3pm']).reshape(1,len(np.array(Test['Cloud3pm'])))
b = np.array(Test['Cloud9am']).reshape(1,len(np.array(Test['Cloud9am'])))
c = a-b
Test['CloudFrac'] = pd.Series(c[0])
a = np.array(Test['Temp3pm']).reshape(1,len(np.array(Test['Temp3pm'])))
b = np.array(Test['Temp9am']).reshape(1,len(np.array(Test['Temp9am'])))
c = a-b
Test['TempFrac'] = pd.Series(c[0])

In [72]:
del skyfall['WindSpeed9am'], skyfall['Humidity9am'], skyfall['Pressure9am'], skyfall['Cloud9am'], skyfall['Temp9am']

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [73]:
del Test['WindSpeed9am'], Test['Humidity9am'], Test['Pressure9am'], Test['Cloud9am'], Test['Temp9am']

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [74]:
del skyfall['RainToday'], Test['RainToday']

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2895: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [75]:
cols = skyfall.columns

In [76]:
np.array(cols).shape

(119,)

In [77]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

skyfall = scaler.fit_transform(skyfall)

Test = scaler.transform(Test)

In [78]:
skyfall = pd.DataFrame(skyfall, columns=np.array(cols))
Test = pd.DataFrame(Test, columns=np.array(cols))

In [79]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression


# instantiate the model
logreg = LogisticRegression(solver='liblinear', random_state=0)


# fit the model
logreg.fit(skyfall, y)

LogisticRegression(random_state=0, solver='liblinear')

In [80]:
y_pred_test = logreg.predict(Test)

In [81]:
len(np.where(y_pred_test !=0)[0])

6619

In [82]:
y_pred_train = logreg.predict(skyfall)

In [83]:
from sklearn.metrics import confusion_matrix

print('Train metrics')
print(confusion_matrix(y, y_pred_train))

Train metrics
[[75382  4152]
 [11142 11146]]


In [84]:
answer = pd.DataFrame({'ID': np.arange(43638),'RainTomorrow': y_pred_test})

In [85]:
answer

,ID,RainTomorrow
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
...,...,...
43633,43633,0
43634,43634,0
43635,43635,0
43636,43636,1


In [86]:
answer.to_csv('AUSout7.csv',index=False)

первый нормальный рез-тат - 6613, второй - 6614, третий - 6615; 6626 -- уже плохо!

In [87]:
from sklearn.metrics import confusion_matrix

print('Train metrics')
print(confusion_matrix(y_train, y_pred_train))

Train metrics


NameError: name 'y_train' is not defined